In [2]:
import pandas as pd
import numpy as np

In [3]:
train_df = pd.read_csv("../stand_pca_Data/train_inpca.csv")
val_df = pd.read_csv("../stand_pca_Data/val_inpca.csv")
test_df = pd.read_csv("../stand_pca_Data/test_inpca.csv")



In [4]:
#functions
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [5]:
train_df.tail()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
113837,-7.883762,4.809679,-6.468902,-0.912184,-3.396197,-1.309048,-3.745595,4.158315,4.343117,7.547046,...,-0.702149,4.645009,-1.826741,-2.257129,1.329637,-1.876749,3.500242,0.469197,3.838660,0.732738
113838,1.027094,-0.023676,-0.644719,1.036438,-0.002088,-0.480674,0.133451,-0.176602,0.604812,0.106014,...,-0.365731,-0.099770,0.264429,0.851902,-0.133605,-0.100048,0.576507,-0.954362,0.029220,0.053592
113839,-0.274876,0.243345,0.210028,-0.248986,0.216420,-1.048178,0.703560,-0.232607,0.106583,-0.288778,...,-0.378218,-0.120563,-0.360048,-0.740484,0.394751,0.633848,-1.083416,0.209637,0.124229,-0.074098
113840,-0.159448,0.316519,-0.464233,-0.405868,0.849618,-0.219923,0.583132,0.014252,-0.183242,-0.661515,...,0.144594,0.031030,-0.515184,-1.140760,0.055808,0.201067,-0.834412,0.224812,0.303160,-0.249796
113841,-6.855427,5.459584,-7.040317,-0.992784,-4.021712,-2.070938,-4.457521,5.924328,1.657972,3.743503,...,-0.796906,1.833663,-0.008867,0.151082,1.657797,-0.736546,2.826693,0.428055,1.770305,0.898151


In [6]:
train_x = train_df[:]
val_x = val_df.drop('Class',axis=1)
val_y = val_df['Class']

In [7]:
from sklearn.ensemble import IsolationForest 
from sklearn.metrics import classification_report,f1_score

val_contamination = val_y.mean()
print(val_contamination)


0.0010540369615627855


In [11]:
colnames = ['V12','V17','V18','V19','V9','V14']
colnames = val_x.columns

In [12]:


model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
model.fit(train_x.loc[:,colnames])

/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.0010540369615627855, max_samples=113842,
                n_estimators=125, random_state=42)

In [13]:
pred = model.predict(val_x.loc[:,colnames])
pred = get_pred_label(pred)

print(classification_report(pred,val_y))
print(f1_score(val_y, pred, average='macro'))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28442
           1       0.07      0.10      0.08        20

    accuracy                           1.00     28462
   macro avg       0.53      0.55      0.54     28462
weighted avg       1.00      1.00      1.00     28462

0.539595597285227
